## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-17-08-39-33 +0000,bbc,Badenoch at odds with Jenrick over 'broken' Br...,https://www.bbc.com/news/articles/cd0y723mr3eo...
1,2026-01-17-08-26-23 +0000,bbc,The one measure that can tell us a lot about t...,https://www.bbc.com/news/articles/c150leql9pgo...
2,2026-01-17-08-13-00 +0000,wsj,Xi’s Enforcers Punished Nearly a Million in 20...,https://www.wsj.com/world/china/xis-enforcers-...
3,2026-01-17-08-08-26 +0000,bbc,Japanese mayor apologises after calling staff ...,https://www.bbc.com/news/articles/czejwl2z80lo...
4,2026-01-17-07-29-34 +0000,bbc,Cafe offers free breakfasts for vulnerable people,https://www.bbc.com/news/articles/c9wxywyrq5jo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
131,trump,64
37,ice,22
348,iran,20
51,new,16
75,president,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
149,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...,128
160,2026-01-16-20-23-34 +0000,nypost,Trump admin slams LA Mayor Karen Bass after sh...,https://nypost.com/2026/01/16/us-news/trump-ad...,120
102,2026-01-16-23-08-00 +0000,wsj,President Trump signaled that he might keep Na...,https://www.wsj.com/economy/central-banking/tr...,120
281,2026-01-16-12-48-22 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/16/us-news/venezuel...,120
25,2026-01-17-04-00-00 +0000,wsj,"Entering the second year of his second term, P...",https://www.wsj.com/politics/policy/over-two-w...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
149,128,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...
63,75,2026-01-17-00-59-04 +0000,cbc,U.S. Justice Department probing Minnesota Gov....,https://www.cbc.ca/news/world/tim-walz-jacob-f...
281,56,2026-01-16-12-48-22 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/16/us-news/venezuel...
314,50,2026-01-16-10-43-00 +0000,wsj,"With Military Action on Hold, the U.S. Puts Ne...",https://www.wsj.com/world/middle-east/with-mil...
137,46,2026-01-16-21-24-32 +0000,cbc,Here's what you need to know about the Canada-...,https://www.cbc.ca/news/politics/what-is-in-ca...
337,44,2026-01-16-09-00-00 +0000,cbc,Federal government loses Emergencies Act appea...,https://www.cbc.ca/news/politics/convoy-protes...
102,44,2026-01-16-23-08-00 +0000,wsj,President Trump signaled that he might keep Na...,https://www.wsj.com/economy/central-banking/tr...
103,38,2026-01-16-22-59-26 +0000,latimes,"In San Francisco, Newsom rails against propose...",https://www.latimes.com/california/story/2026-...
14,34,2026-01-17-05-24-01 +0000,nypost,Feds in Minneapolis immigration operation can’...,https://nypost.com/2026/01/17/us-news/feds-in-...
140,33,2026-01-16-21-19-26 +0000,nypost,Trump and governors unveil plan to fight AI el...,https://nypost.com/2026/01/16/us-news/trump-an...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
